In [3]:
# RDF Generator

# Ontology


In [4]:
import os
import re
import gzip
import shutil
import unicodedata
from datetime import datetime

from SPARQLWrapper import RDF
from rdflib import Namespace, URIRef, Graph, Literal
from rdflib.namespace import RDF

In [5]:
file_name = 'mitteilungsblatt-names-alpha-01.ttl'
working_path = "./"
output_path = "/data/judaicalink/dumps/mb/current/"

graph = Graph()

skos = Namespace("http://www.w3.org/2004/02/skos/core#")
jl = Namespace("http://data.judaicalink.org/ontology/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
gndo = Namespace("http://d-nb.info/standards/elementset/gnd#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
edm = Namespace("http://www.europeana.eu/schemas/edm/")
dc = Namespace("http://purl.org/dc/elements/1.1/")

graph.bind('skos', skos)
graph.bind('foaf', foaf)
graph.bind('jl', jl)
graph.bind('gndo', gndo)
graph.bind('owl', owl)
graph.bind('edm', edm)
graph.bind('dc', dc)


In [6]:
import pandas as pd

In [24]:
from os import walk

columns=['first name', 'last name', 'title', 'jub', 'issue_date', 'other', 'nachruf/tode','Geburt/Tod', 'ID']

df = pd.DataFrame()

path = './results/csv'
def read_csv_files(path):

    df_frames = []

    for (dirpath, dirnames, filenames) in walk(path):
        for file in filenames:
            print(os.path.join(path, file))
            df = pd.read_csv(os.path.join(path, file),
                             #names=columns,
                             delimiter=';',
                             encoding='utf-8',
                             engine='python',
                             encoding_errors='ignore')
            df_frames.append(df)
    return pd.concat(df_frames)

df = read_csv_files(path=path)

df.sample(10)

./results/csv/2_MB_Nachtraege_1933-1961.csv
./results/csv/3_Namensregister_Jubilaen_Nachrufe_1962-1965_v02_UW.csv
./results/csv/8_Register_MB_1975-1977.pdf.csv
./results/csv/10_MB-Personalia_1981-1982.pdf.csv
./results/csv/6_Namensregister_MB_1970-1973.pdf.csv
./results/csv/7_MB-Jubilaen-Register1973-1974.pdf.csv
./results/csv/1_Namensregister_MB_1933-1961_Geburtstage_etc.pdf.csv
./results/csv/12_MB-RegisterJuni1985-Juli1990.pdf.csv
./results/csv/9_GeburtstageNachrufeKulturnotizen_1978-1980.pdf.csv
./results/csv/5_Namensregister_Jubilaen_Nachrufe_1968-1969.pdf.csv
./results/csv/4_Namensregister_Jubilaen_Nachrufe_1966-1968.pdf.csv
./results/csv/11_MB-JubilaenBiographienRezensionen1983-85.pdf.csv


,first name,last name,issue_date,other,title,nachruf/tode,jub,Geburt/Tod,ID
146,Gershom,Schölern,21.4.61,Flusser),-,-,-,-,MBNR.2.PER.146
180,Kurt,kay,13.83.76,80 HIT,-,-,-,-,MBNR.8.PER.180
702,Salman,levontin,28.6.40,David :achruf,-,-,-,-,MBNR.1.PER.702
272,Marianne,Littmann,-,Todanz„9.8.63,-,-,-,-,NaN
8,Paula,Arnold,13.9.68,Nachruf(KL),-,-,-,-,MBNR.5.PER.8
1277,Jizchak,Volcani(Wilkansky),-,Prof Jizchak,Prof.,-,-,-,MBNR.1.PER.1277
153,Fritzi,Hassary‚,-,Todesnachricht 21269,-,-,-,-,MBNR.5.PER.153
676,Helmut,Lange,-,(NewYork) 46/1950 Tod,-,-,-,-,MBNR.1.PER.676
1256,Georg,Tuchler,-,"13,8.54 iachruf",-,-,-,-,MBNR.1.PER.1256
35,Hans,Beyth,21.12.67,20.Todestag,-,-,-,-,MBNR.4.PER.35


In [ ]:
# Generate the rdf

In [ ]:
def compress_ttl(file_name):
    """
    Compress the ttl file.
    returns: compressed ttl file.
    """
    # compress the ttl file
    try:
        with open(file_name, 'rb') as f_in:
            with gzip.open(file_name + '.gz', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    except Exception as e:
        print("Could not compress file. Error: ", e)


In [1]:
def move_ttl_file(file_name):
    """
    Move the ttl file to the correct folder.
    returns: the path of the ttl file.
    """
    # move the ttl file to the correct folder
    try:
        shutil.move(file_name, output_path)
    except Exception as e:
        print("Could not move file. Error: ", e)


In [2]:
def clean_url_string(string):
    """
    Clean the name of a person.
    returns: cleaned name.
    """

    string = string.strip()
    string = string.replace('\'', '')
    string = string.replace('"', '')
    string = string.replace(',', '_')
    string = string.replace('<<', '')
    string = string.replace('>>', '')
    string = string.replace('|', '_')
    string = string.replace(' ', '')
    string = string.replace('<', '_')
    string = string.replace('>', '_')
    string = string.replace('.', '')
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace('(', '')
    string = string.replace(')', '')
    string = string.replace('{', '')
    string = string.replace('}', '')
    string = string.replace('#', '')
    string = string.replace('-', '')
    string = string.replace('?', '')

    string =  unicodedata.normalize('NFKD', string).encode('ascii', 'ignore')
    return string

In [3]:
def generate_rdf(df):
    skos = Namespace("http://www.w3.org/2004/02/skos/core#")
    jl = Namespace("http://data.judaicalink.org/ontology/")
    foaf = Namespace("http://xmlns.com/foaf/0.1/")
    gndo = Namespace("http://d-nb.info/standards/elementset/gnd#")
    owl = Namespace("http://www.w3.org/2002/07/owl#")
    edm = Namespace("http://www.europeana.eu/schemas/edm/")
    dc = Namespace("http://purl.org/dc/elements/1.1/")
    dcterms = Namespace("http://purl.org/dc/terms/")

    graph.bind('skos', skos)
    graph.bind('foaf', foaf)
    graph.bind('jl', jl)
    graph.bind('gndo', gndo)
    graph.bind('owl', owl)
    graph.bind('edm', edm)
    graph.bind('dc', dc)



SyntaxError: incomplete input (1568686394.py, line 2)